In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

# RunnablePassthrough

* It does not do anything to the input data
* Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()    

In [ ]:
chain.invoke("Bappy")

# RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda
* Let's define a very simple function to create Russian lastnames:

In [1]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"
   

In [ ]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)  

# RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    *  operation_a will use RunnablePassthrough.
    *  operation_b will use RunnableLambda with the russian_lastname function

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)     
    }
)

In [ ]:
chain.invoke("Bappy")

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate   

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()   

In [ ]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [ ]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),     
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser  


In [ ]:
chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})

# Let's see a more advanced use of RunnableParallel

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser   
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings   

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()}) 
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is dswithbappy about?")